In [1]:
import pandas as pd 

#from quartile_v2 import *
from crossref.restful import Works, Etiquette
from scihub_blastoise import SciHub_HydroPump

import pickle

## PART 0: Jobs to be Done!

There are 3 main topics, we need to focus on:

1- Standardizing journal dfs & their pipelines

2- Creating Q2 journal df + CR_count + SH_Results

3- Merging Q1 article df to journals ?


## PART 1: Journal df & Standardization

In [106]:
with open("retr_q1", "rb") as fp:
    retr_q1 = pickle.load(fp)

In [107]:
retr_q1

Sbj_Area  \
Issn                 Year                                                      
(00011541, 15475905) 2010  {'Biochemistry, Genetics and Molecular Biology...   
                     2011  {'Chemical Engineering': 1, 'Environmental Sci...   
                     2012  {'Chemical Engineering': 1, 'Environmental Sci...   
                     2013  {'Biochemistry, Genetics and Molecular Biology...   
                     2014  {'Biochemistry, Genetics and Molecular Biology...   
...                                                                      ...   
(8756758X, 14602695) 2017         {'Materials Science': 1, 'Engineering': 2}   
                     2018         {'Materials Science': 1, 'Engineering': 2}   
                     2019         {'Materials Science': 1, 'Engineering': 2}   
                     2020         {'Materials Science': 1, 'Engineering': 2}   
(8756971X, 19436270) 2010  {'Agricultural and Biological Sciences': 1, 'M...   

                          CR_Results  SH_Results  CR_Count  Total_Docs  
Issn                 Year                                               
(00011541, 15475905) 2010       True        True       351         294  
                     2011       True       False       361         315  
                     2012       True       False       259         347  
                     2013       True        True       343         422  
                     2014       True        True       376         359  
...                              ...         ...       ...         ...  
(8756758X, 14602695) 2017       True        True       194         170  
                     2018       True        True       201         199  
                     2019       True        True       202         210  
                     2020       True        True       239         221  
(8756971X, 19436270) 2010       True       False        74          69  

[74421 rows x 5 columns]

### PART 1a: Getting Q1 dfs ready

Scimago excels contain a lot of important metadata info on q1 journals. Will create a q1_jrnl_meta df to check these columns and finalize which ones will be saved for later use.

In [58]:
q1_jrnl_meta = quar_df_creator(1)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


Need a function to simplify the journal dfs!

In [ ]:
def jrnl_id_creator(n_id, df):
     return [f"{n_id}_{x}" for x in range(len(df))]

In [48]:
def jrnl_meta_simplifier(jrnl_df, n_id):

    jrnl_df.rename(columns={"Issn":"ISSN"}, inplace=True)
    jrnl_df["ISSN"] = jrnl_df.ISSN.map(tuple)
    
    jrnl_df["Year"] = jrnl_df.Year.astype(int)

    jrnl_df = jrnl_df[jrnl_df.Total_Docs>20]

    jrnl_df.sort_values("ISSN", inplace=True)

    jrnl_df.insert(loc=0, column='ID', value=jrnl_id_creator(n_id,jrnl_df))

    jrnl_df.set_index("ID", inplace=True)

    return jrnl_df[["ISSN", "Year", "Title", "Sbj_Area", "Total_Docs", 'Country', 'Region', 'Publisher', 'Coverage', 'SJR', 'H index', 'Total Docs. (3years)', 'Total Refs.', 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)', 'Ref. / Doc.']]

Added to quartile_v2.py

In [126]:
q1_jrnl_df = jrnl_meta_simplifier(q1_jrnl_meta)

q1_jrnl_df is ready! However, for Q1 loop only, as we alreadyhave all the other relevant information (abt. CR & SH), we can simply merge with retr_q1 to get final dataset. 

However, for Q2 journals we need to follow the actual pipeline in this step

In [127]:
# Merging columns for q1_jrnl_df

q1_jrnl_df.set_index(["ISSN","Year"], inplace=True)

retr_q1_merge = retr_q1.reset_index().rename(columns={"Issn":"ISSN"}).set_index(["ISSN","Year"])[["CR_Results","SH_Results","CR_Count"]]

q1_jrnl_df = pd.merge(q1_jrnl_df,retr_q1_merge, how="left", left_index=True, right_index=True, suffixes=("_old", "_new"))

In [ ]:
# Making ID column the index for data standardization:

q1_jrnl_df = q1_jrnl_df.reset_index().set_index("ID")

q1_jrnl_df is now complete & ready for pivot table'ing & sample count calculations. 

For this, we need to modify explade_v2() as there were some errors that were not noticed during earlier notebooks. The code below will therefore is unecessary once these changes are implemented to quartile_v2.py

#### quartile_v2() corrections:

In [144]:
def explade(countdf):
    # No need to extract ISSNs as CrossRef func. accepts lists as argument
    count1_df = countdf[countdf["Sbj_Area"].map(len) == 1]
    count1_df["Sbj_Area"] = count1_df["Sbj_Area"].map(list).str[0]
    count2_df = countdf[countdf["Sbj_Area"].map(len) > 1]
    
    newcount_list = []

    for index, row in count2_df.iterrows():
        issn = row["ISSN"]
        year = row["Year"] 
        s_a = row["Sbj_Area"]
        #cr_ = row["CR_Results"]
        sh_ = row["SH_Results"]
        #crc = row["CR_Count"]
        tot_docs = row["Total_Docs"]

        tot_sa_count = sum(list(s_a.values()))

        rmndr = tot_docs % tot_sa_count
        rmndrchoice = Counter(choices(list(s_a.keys()), weights=list(s_a.values()), k= rmndr))

        for sbj, val in s_a.items():
            count = tot_docs*val/tot_sa_count

            if sbj in list(rmndrchoice.keys()):
                #to_append = [issn, year, sbj, cr_, sh_, crc, int(count+rmndrchoice[sbj])]
                to_append = [issn, year, sbj, sh_, int(count+rmndrchoice[sbj])]
                
            else:
                to_append = [issn, year, sbj, sh_, int(count)]
            newcount_list.append(to_append)

    expladed_df = count1_df.append(pd.DataFrame(newcount_list, columns=count1_df.columns), ignore_index=True)
    return expladed_df


def retr_ready(q_df):
    # Step 1: Create a dataframe that only has the values necessasry for the final pivot table
    #q_main = q1_df[["ISSN", "Year", "Sbj_Area", "CR_Results", "SH_Results", "CR_Count", "Total_Docs"]]
    q_main = q_df[["ISSN", "Year", "Sbj_Area", "SH_Results", "Total_Docs"]]

    #q1_df = q1_df[q1_df.Sbj_Area.map(len) < q1_df.Total_Docs]
    q_main = explade(q_main)

    # Step 2: Create the CONSTANT sample_count pivot table from pivotq1_full

    sa_pivot_full = pd.pivot_table(data=q_main, columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

    sa_pivot_meta_only = pd.pivot_table(data=q_main[q_main.SH_Results == True], columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

    return q_main, sa_pivot_full, sa_pivot_meta_only

In [145]:
df1, df2, df3 = retr_ready(q1_jrnl_df)

The dfs above show that everything is working as intended and ready for futher steps. Will create an ID function & mpa it. This step may not be necessarily required, BUT still might prove useful in later steps. So it will be created just in case.

Before moving on, q1_jrnl_df will be SAVEd and DUMPed!

In [148]:
%store q1_jrnl_df

with open("q1_jrnl_df","wb") as fp:
    pickle.dump(q1_jrnl_df, fp)

Stored 'q1_jrnl_df' (DataFrame)


### PART 1b: Getting Q2 dfs ready

In [2]:
q2_jrnl_meta = quar_df_creator(2)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [49]:
q2_jrnl_df = jrnl_meta_simplifier(q2_jrnl_meta,2)

In [112]:
#Converting ISSN column to list to be used for CrossRef API:

q2_jrnl_df["ISSN"] = q2_jrnl_df.ISSN.map(list)

### PART 1c: CR_Count for q2_jrnl_df

In [ ]:
# Full Loop Run (for q2_jrnl_df):

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', 'oguzkokes@gmail.com')
ogi_works = Works(etiquette=my_etiquette)

q2_dict = q2_jrnl_df[["ISSN","Year"]].to_dict("index")


full_meta_list = []
useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'reference', 'container-title', 'issued', 'ISSN', 'subject']


for key in q2_dict:
    issn = q2_dict[key]["ISSN"]
    year = q2_dict[key]["Year"]

    jrnl_count = ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").count() 

    q2_jrnl_df.loc[key,"CR_Count"] = jrnl_count

    if jrnl_count >1:
        loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(2).select(useful_cols)] 
        loop_meta = [dict(item, jrnl_ID=key) for item in loop_meta]
        
        full_meta_list.extend(loop_meta)

with open("q2_jrnl_df","wb") as fp:
    pickle.dump(q2_jrnl_df, fp)

with open("full_meta_list","wb") as fp:
    pickle.dump(full_meta_list, fp)

In [116]:
with open("q2_jrnl_df","wb") as fp:
    pickle.dump(q2_jrnl_df, fp)

In [2]:
with open("q2_jrnl_df", "rb") as fp:
    q2_jrnl_df = pickle.load(fp)

In [3]:
q2_jrnl_sum = q2_jrnl_df[:17000]
q2_jrnl_cet = q2_jrnl_df[17000:34000]
q2_jrnl_ogi = q2_jrnl_df[34000:]

In [4]:
# Ogi Loop Run (for q2_jrnl_df):

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', 'oguzkokes@gmail.com')
ogi_works = Works(etiquette=my_etiquette)

q2_dict = q2_jrnl_ogi[["ISSN","Year"]].to_dict("index")


ogi_meta_list = []
useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'reference', 'container-title', 'issued', 'ISSN', 'subject']


for key in q2_dict:
    issn = q2_dict[key]["ISSN"]
    year = q2_dict[key]["Year"]

    jrnl_count = ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").count() 

    q2_jrnl_ogi.loc[key,"CR_Count"] = jrnl_count

    if jrnl_count >1:
        loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(2).select(useful_cols)] 
        loop_meta = [dict(item, jrnl_ID=key) for item in loop_meta]
        
        ogi_meta_list.extend(loop_meta)



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
# As there was an unexpected error toward the end of the loop, the 2 variables will be saved in case & the run will be started again. (from ID = 2_40256)

with open("q2_jrnl_ogi","wb") as fp:
    pickle.dump(q2_jrnl_ogi, fp)

with open("ogi_meta_list","wb") as p:
    pickle.dump(ogi_meta_list, p)


In [27]:
q2_dict_2 = q2_jrnl_ogi[12653:][["ISSN","Year"]].to_dict("index")


#ogi_meta_list = []
useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'reference', 'container-title', 'issued', 'ISSN', 'subject']


for key in q2_dict_2:
    issn = q2_dict_2[key]["ISSN"]
    year = q2_dict_2[key]["Year"]

    jrnl_count = ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").count() 

    q2_jrnl_ogi.loc[key,"CR_Count"] = jrnl_count

    if jrnl_count >1:
        loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(2).select(useful_cols)] 
        loop_meta = [dict(item, jrnl_ID=key) for item in loop_meta]
        
        ogi_meta_list.extend(loop_meta)



SumCetCet is completed all variables are DUMPed or WHATSAPPed. 

The next step is the SH_Results column. However, the date info gathered during this step should also be saved to avoid duplicate work.

For this reason a custom loop is necessary.

## PART 2: SH_Results for q2_jrnl_df

To get the SH_Results for Q2 articles, we need to create sh_blastoise, as wartortle only works with pdf_urls and not with DOIs.

sh_blastoise is created & tested succesfully in the cells below. 

Next step is to create a SumCetCet loop:
 

In [5]:
with open("q2_jrnl_ogi", "rb") as fp:
    q2_jrnl_ogi = pickle.load(fp)

with open("ogi_meta_list", "rb") as fp:
    ogi_meta_list = pickle.load(fp)

In [3]:
ogi_meta_df = pd.DataFrame(ogi_meta_list)

ogi_jrnl_ID_DOI_dict = ogi_meta_df.groupby("jrnl_ID")["DOI"].apply(list)

In [14]:
ogi_meta_df.sample(1)

,publisher,published-print,DOI,is-referenced-by-count,title,author,published-online,reference,container-title,issued,references-count,ISSN,subject,jrnl_ID
7278,Informa UK Limited,"{'date-parts': [[2021, 2, 7]]}",10.1080/17517575.2020.1844305,2,[Adaptive monitoring for autonomous vehicles u...,[{'ORCID': 'http://orcid.org/0000-0002-4631-58...,"{'date-parts': [[2020, 11, 11]]}","[{'doi-asserted-by': 'publisher', 'key': 'cit0...",[Enterprise Information Systems],"{'date-parts': [[2020, 11, 11]]}",39,"[1751-7575, 1751-7583]","[Information Systems and Management, Computer ...",2_38396


In [2]:
# Testing Blastoise:

sh_h = SciHub_HydroPump()

testa, testb, testc = sh_h.get_dates_kws("10.1080/17517575.2020.1844305")

In [3]:
testa

'http://sci-hub.st/downloads/2020-12-17/c6/zavala2020.pdf#navpanes=0&view=FitH'

In [4]:
testb

['25 September 2020', '27 October 2020']

In [18]:
testc

[('autonomous', 0.3739),
 ('monitoring', 0.3523),
 ('vehicles', 0.3254),
 ('adaptive', 0.3194),
 ('sensor', 0.2808),
 ('runtime', 0.2522),
 ('sensors', 0.2504),
 ('systems', 0.2452),
 ('data', 0.2382),
 ('dynamic', 0.2367)]

In [19]:
# Testing the loop:

ogi_meta_df = pd.DataFrame(ogi_meta_list).sample(10)
ogi_jrnl_ID_DOI_dict = ogi_meta_df.groupby("jrnl_ID")["DOI"].apply(list).to_dict()



In [6]:
# SH_Results Ogi Run:
#  
sh_h = SciHub_HydroPump()

ogi_meta_df = pd.DataFrame(ogi_meta_list)
ogi_jrnl_ID_DOI_dict = ogi_meta_df.groupby("jrnl_ID")["DOI"].apply(list).to_dict()

q2_date_dict_ogi = list() 

for key, val in ogi_jrnl_ID_DOI_dict.items():
    for doi in val:
        pdf_url, date, kw = sh_h.get_dates_kws(doi)

        if type(date) == list:
            q2_jrnl_ogi.loc[key, "SH_Results"] = True
            q2_date_dict_ogi.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))
            break
        else:
            
            pass



In [11]:
# As there was an unexpected error toward the end of the loop, the 2 variables will be saved in case & the run will be started again. (from ID = 2_40256)

with open("q2_date_dict_ogi","wb") as fp:
    pickle.dump(q2_date_dict_ogi, fp)

with open("q2_jrnl_ogi_SH","wb") as p:
    pickle.dump(q2_jrnl_ogi, p)


In [15]:
q2_jrnl_ogi[q2_jrnl_ogi.SH_Results==True]

,ISSN,Year,Title,Sbj_Area,Total_Docs,Country,Region,Publisher,Coverage,SJR,H index,Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,CR_Count,SH_Results
ID,,,,,,,,,,,,,,,,,,,
2_34006,"[16156102, 0033183X]",2011,Protoplasma,"{'Medicine': 1, 'Agricultural and Biological S...",76,Austria,Western Europe,Springer-Verlag Wien,"1926-1943, 1949-2020","0,649",74,208,4588,349,189,"1,74","60,37",112.0,True
2_34007,"[16156102, 0033183X]",2010,Protoplasma,"{'Medicine': 1, 'Agricultural and Biological S...",117,Austria,Western Europe,Springer-Verlag Wien,"1926-1943, 1949-2020","0,743",74,157,6943,276,144,"1,41","59,34",156.0,True
2_34009,"[16156714, 14345293]",2015,Journal of Orofacial Orthopedics,"{'Medicine': 1, 'Dentistry': 1}",42,Germany,Western Europe,Urban und Vogel,1996-2020,"0,615",42,129,718,117,119,"0,67","17,10",44.0,True
2_34010,"[16156714, 14345293]",2018,Journal of Orofacial Orthopedics,"{'Medicine': 1, 'Dentistry': 2}",38,Germany,Western Europe,Urban und Vogel,1996-2020,"0,623",42,137,1396,157,133,"1,04","36,74",43.0,True
2_34011,"[16156714, 14345293]",2020,Journal of Orofacial Orthopedics,"{'Medicine': 1, 'Dentistry': 2}",54,Germany,Western Europe,Urban und Vogel,1996-2020,"0,623",42,123,1529,183,108,"1,52","28,31",63.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2_51425,"[87567938, 15206033]",2010,Biotechnology Progress,"{'Biochemistry, Genetics and Molecular Biology...",210,United States,Northern America,Wiley-Blackwell,1985-2020,"0,997",129,593,6611,1630,590,"2,45","31,48",151.0,True
2_51426,"[87567938, 15206033]",2012,Biotechnology Progress,"{'Biochemistry, Genetics and Molecular Biology...",186,United States,Northern America,Wiley-Blackwell,1985-2020,"0,831",129,622,7318,1675,618,"2,25","39,34",165.0,True
2_51427,"[87567938, 15206033]",2017,Biotechnology Progress,"{'Biochemistry, Genetics and Molecular Biology...",168,United States,Northern America,Wiley-Blackwell,1985-2020,"0,676",129,523,7275,1197,519,"2,12","43,30",181.0,True


In [8]:
pd.DataFrame(q2_date_dict_ogi)

,DOI,direct_url,Dates,Keywords
0,10.1007/s00709-011-0335-9,https://sci.bban.top/pdf/10.1007/s00709-011-03...,"[7 August 2011, 5 October 2011]","[(transresveratrol, 0.453), (resveratrolpteros..."
1,10.1007/s00709-009-0098-8,https://sci.bban.top/pdf/10.1007/s00709-009-00...,"[11 August 2009, 10 December 2009]","[(plants, 0.3844), (arabidopsis, 0.3839), (bio..."
2,10.1007/s00056-014-0276-y,https://sci.bban.top/pdf/10.1007/s00056-014-02...,"[21 February 2014, 7 August 2015]","[(orthodontics, 0.5551), (orthodontic, 0.5051)..."
3,10.1007/s00056-018-0156-y,https://sci.bban.top/pdf/10.1007/s00056-018-01...,"[5 February 2018, 8 August 2018]","[(mandibulares, 0.554), (mandibular, 0.533), (..."
4,10.1007/s00056-020-00224-8,https://sci.bban.top/pdf/10.1007/s00056-020-00...,"[2 January 2019, 21 January 2020]","[(orthodontics, 0.5295), (maxillary, 0.5003), ..."
...,...,...,...,...
7269,10.1002/btpr.374,https://sci.bban.top/pdf/10.1002/btpr.374.pdf#...,"[Jun. 25, 2009, None]","[(cellcelectortm, 0.4886), (cellcelector, 0.47..."
7270,10.1002/btpr.1668,https://sci.bban.top/pdf/10.1002/btpr.1668.pdf...,"[July 5, 2012, None]","[(chondrocytes, 0.5303), (cartilages, 0.5177),..."
7271,10.1002/btpr.2510,https://sci.bban.top/pdf/10.1002/btpr.2510.pdf...,"[Apr 05, 2017, Jun 05, 2017]","[(protozoa, 0.3711), (petrochemical, 0.3516), ..."
7272,10.1002/btpr.2080,https://sci.bban.top/pdf/10.1002/btpr.2080.pdf...,"[Dec 02, 2014, Mar 05, 2015]","[(chromatography, 0.4931), (bioseparations, 0...."


___REMAINING TASKS AFTER THIS RUN:___

* Q1 artcl & jrnl matching (date_df & q1_jrnl_df)

* calculating sample size for both Q1 & Q2 jrnls & artcls using pivots

* complete q1 & q2 meta_dfs -> date_dfs

* complete Scopus API run to get date_dfs -> artcl_dfs


___Important Variables:___

__FOR Q1:__

* date_df_ok_final -> date_df for Q1 (DOES NOT HAVE JRNL_ID)
* q1_jrnl_df -> jrnl_df for Q1 (DOES NOT HAVE SH_DATE)

__FOR Q2:__

* q2_jrnl_ogi (SumCetCet) -> BEFORE SH RUN, jrnl_df for Q2 (DOES NOT HAVE SH_RESULTS) -> NOT IMPORTANT! BACKUP'ED
* q2_jrnl_ogi_SH (SumCetCet) -> AFTER SH RUN, jrnl_df for Q2
* ogi_meta_df (SumCetCet) -> BEFORE SH RUN, meta_df for Q2 (DOES NOT HAVE URL, DATE, KEYWORDS)
* q2_date_dict_ogi (SumCetCet) -> AFTER SH_RUN, NOT date_df for Q2 (ONLY CONTAINS URL, DATE, KEYWORDS)


___Next Steps:___

__FOR Q1:__

* transform date_df_ok_final -> q1_date_df => add JRNL_ID column
* add SH_DATES to q1_jrnl_df

__FOR Q2:__
* merge SumCetCet for:

    q2_jrnl_ogi_SH, - __DONE__ - q2_jrnl_df

    ogi_meta_df - __DONE__ - q2_meta_df
    
    q2_date_dict_ogi - __DONE__
    
* merge q2_meta_df + q2_date_dict_df -> q2_date_df - __DONE__ - q2_meta_df


__Extra Steps:__

* need published column for q2_date_df & check if any other modifications necessary

--- 
SumCetCet run is completed however, it is realized that the SH_Results are True for only date found records. Some journals that may indeed date info, BUT was False due to another error message are not distinguishable from "not found in Scihub" errors. 

To avoid time loss and due to the fact that the rety decorators allow jrnls & artcls several retries effectively, any False record can be skipped.

Continuing with the next steps:

### PART 3: THE COMBINATION


In [29]:
# 1- q2_jrnl_df

with open("q2_jrnl_sum_SH", "rb") as fp:
    q2_jrnl_sum_SH = pickle.load(fp)

with open("q2_jrnl_cet_SH", "rb") as fp:
    q2_jrnl_cet_SH = pickle.load(fp)

with open("q2_jrnl_ogi_SH", "rb") as fp:
    q2_jrnl_ogi_SH = pickle.load(fp)


q2_jrnl_df = pd.concat([q2_jrnl_sum_SH,q2_jrnl_cet_SH,q2_jrnl_ogi_SH])

with open("q2_jrnl_df","wb") as p:
    pickle.dump(q2_jrnl_df, p)

In [17]:
# 2- q2_meta_df 

with open("sum_meta_list", "rb") as fp:
    sum_meta_list = pickle.load(fp)

with open("cet_meta_list", "rb") as fp:
    cet_meta_list = pickle.load(fp)

ogi_meta_df.rename(columns={"jrnl_ID": "JRNL_ID"}, inplace=True)

q2_meta_df = pd.concat([pd.DataFrame(sum_meta_list),pd.DataFrame(cet_meta_list),ogi_meta_df], ignore_index=True)

with open("q2_meta_df","wb") as p:
    pickle.dump(q2_meta_df, p)


In [ ]:
# 3- q2_date_dict_df

with open("q2_date_dict_sum", "rb") as fp:
    q2_date_dict_sum = pickle.load(fp)

with open("q2_date_dict_cet", "rb") as fp:
    q2_date_dict_cet = pickle.load(fp)

with open("q2_date_dict_ogi", "rb") as fp:
    q2_date_dict_ogi = pickle.load(fp)


q2_date_dict_df = pd.concat([pd.DataFrame(q2_date_dict_sum), pd.DataFrame(q2_date_dict_cet), pd.DataFrame(q2_date_dict_ogi)],ignore_index=True)

with open("q2_date_dict_df","wb") as p:
    pickle.dump(q2_date_dict_df, p)

In [56]:
# 4- q2_date_df:

q2_date_df = pd.merge(q2_meta_df, q2_date_dict_df, how="left", on="DOI")

with open("q2_date_df","wb") as p:
    pickle.dump(q2_date_df, p)

---

___Standardizing date_df(s):___

__1- q2_date_df:__

* add published column - __DONE__
* remove refereence column - __DONE__
* reorder columns for easier analysis - __DONE__
* set DOI as index - __DONE__
* drop no_date articles - __DONE__

__2- q1_date_df:__

* filter & create from date_df_final_ok- __DONE__
* re-modernize col  reference-count -> references-count / New_dates -> Dates / Results -> direct_url
* add JRNL_ID col

__3- q1_jrnl_df:__
* add JRNL_ID - DONE



In [4]:
# 1- q2_date_df:

with open("q2_date_df", "rb") as fp:
    q2_date_df = pickle.load(fp)

def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]


q2_date_df.loc[:,"published"] = q2_date_df.apply(earlier_date_part, axis=1)

q2_date_df = q2_date_df[["DOI", "JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]

q2_date_df.set_index("DOI", inplace=True)

q2_date_df = q2_date_df[q2_date_df.direct_url.notna()]

with open("q2_date_df","wb") as p:
    pickle.dump(q2_date_df, p)

In [67]:
# 2- q1_jrnl_df:
# ID column was deleted by mistake, re-created:

def jrnl_id_creator(n_id, df):
     return [f"{n_id}_{x}" for x in range(len(df))]

q1_jrnl_df.insert(loc=0, column='ID', value=jrnl_id_creator(1,q1_jrnl_df))

q1_jrnl_df = q1_jrnl_df.reset_index().set_index("ID")

q1_jrnl_df = q1_jrnl_df[q2_jrnl_df.columns.tolist()]

with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

At this point:

* q1_jrnl_df & q2_jrnl_df should be standardized & have the exact same column info & order

* q2_date_df is ready!

* q1_date_df is MISSING!


---

To create q1_date_df we need to use info from part 5a 

In [121]:
# 3- q1_date_df


q1_date_df = date_df_ok_final[(date_df_ok_final.New_dates.notna()) & (date_df_ok_final.New_dates.map(type) == list)].copy()

q1_date_df.loc[:,"direct_url"] = q1_date_df.Results.map(lambda x: x[0])

#q1_date_df.loc[:,"Old_Dates"] = q1_date_df.Results.map(lambda x: [x[1], x[2]])

# Upon further inspection, it is seen that the New_Dates column is more up-to-date, as it is retrieved on WARTORTLE run! 
# therefore, results can be used only to extract direct_url information & dropped later. Always can be accessed via date_df_ok_final

q1_date_df.rename(columns={"New_dates" : "Dates", "reference-count" : "references-count"}, inplace= True)

q1_date_df = q1_date_df[['ISSN', 'issued', 'container-title', 'publisher', 'title',
       'author', 'subject', 'references-count', 'is-referenced-by-count',
       'published', 'published-print', 'published-online', 'direct_url',
       'Dates', 'Keywords', "Affiliation"]]

with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)


In [ ]:
# JRNL_ID FOR q1_date_df:

# Creating a compatible ISSN clmn:
q1_date_df["ISSN_simp"] = q1_date_df.ISSN.map(lambda x: [y.replace("-", "").strip() for y in x])


# Creating a mapping function:

def id_retr(daterow, jrnldf):
    issn = daterow.ISSN_simp
    year = daterow["issued"]["date-parts"][0][0]
    jrnl_ids = jrnldf[(jrnldf.ISSN.map(lambda x: any(iss in x for iss in issn))) & (jrnldf.Year == year)]["ID"].tolist()
    return jrnl_ids


In [154]:
q1_date_df["JRNL_ID_list"] = q1_date_df.apply(lambda x: id_retr(x,q1_jrnl_df.reset_index()), axis=1)

In [156]:
q1_date_df.JRNL_ID_list.map(len).value_counts()

1    34319
0     1978
2        3
Name: JRNL_ID_list, dtype: int64

After some analysis, it is seen that there are several reasons as to why each artcl did not exactly match with a single journal:

* Different versions of the same journal might share the same ISSN
* jrnl might be filtered from q1_jrnl_df bec. <20 article count
* The year of artcl & jrnl do not exactly match due to data difference

Rather than dealing with these issues individually & the unmatched artcls will be removed from q1_date_df and will be cont. with 34319 artcls

NOTE: The "old" df will be saved as q1_date_df_all in the backups folder in case


In [206]:
q1_date_df = q1_date_df[q1_date_df.JRNL_ID_list.map(len) == 1]

q1_date_df["JRNL_ID"] = q1_date_df.JRNL_ID_list.map(lambda x: x[0])


q1_date_df = q1_date_df[['JRNL_ID', 'ISSN', 'issued', 'container-title', 'publisher', 'title',
       'author', 'subject', 'references-count', 'is-referenced-by-count',
       'published', 'published-print', 'published-online', 'direct_url',
       'Dates', 'Keywords', "Affiliation"]]


with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

At this point:

ALL VARIABLES ARE READY FOR THE NEXT STEP:

JRNL_DFs -> q1_jnrl_df & q2_jrnl_df 

ARTCL_DFs -> q1_date_df & q2_date_df

_Minor Fixes before the next nb:_

* q2_jnrl fillna() for SH_Results



In [238]:
q2_jrnl_df.SH_Results.fillna(False, inplace=True)

with open("q2_jrnl_df","wb") as p:
    pickle.dump(q2_jrnl_df, p)